In [1]:
import json
import os
from pprint import pprint
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import scipy
from scipy.io import wavfile, loadmat, savemat
import sklearn

plt.rcParams['figure.figsize'] = (18, 5)

In [2]:
datadir = 'all_participants/raw_data/'

def get_participant_data(filepath):
    raw = mne.io.read_raw_fif(filepath)
    eeg = raw.to_data_frame()
    events = raw.info['events']
    samples_ids = np.zeros_like(events)
    events_ids = np.zeros_like(events)
    
    for i, item in enumerate(events):
        samples_ids[i] = item['list'][0]
        events_ids[i] = item['list'][2]
    
    samples_ids = np.array([-1] + list(samples_ids))
    eeg['event_ids'] = events_ids.repeat(np.diff(samples_ids))
    return eeg


In [78]:
participants = {}
for _, _, filenames in os.walk(datadir):
    for f in filenames:
        participants[f] = get_participant_data(os.path.join(datadir, f))


Opening raw data file all_participants/raw_data/participant_5.fif...


/var/folders/0f/x7xs6kjs6k7dfph84ymzzlrr0000gn/T/ipykernel_86248/3141000968.py:4: RuntimeWarning: This filename (all_participants/raw_data/participant_5.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(filepath)


Isotrak not found
    Range : 0 ... 3121149 =      0.000 ...  6242.298 secs
Ready.


ValueError: Length of values (3118480) does not match length of index (3121150)

In [95]:
for _, _, filenames in os.walk(datadir):
    for f in filenames:
        if f.startswith('participant_2'):
            raw = mne.io.read_raw_fif(os.path.join(datadir, f))
            print(raw.info)
            eeg = raw.to_data_frame()
            events = raw.info['events']
            p_events = {}
            count = 0
            trial = {}
            for item in events:
                l = item['list']
                if l[2] == 1: print(l)
                if l[2] == 3 or l[2] == 4: continue
                if l[2] == 5 and len(trial) == 0:
                    trial['start'] = l[0]
                elif l[2] == 2 and len(trial) == 2:
                    trial['end'] = l[0]
                    p_events[count] = trial
                    count += 1
                    trial = {}
                elif len(trial) == 1:
                    trial['event'] = l[2]
                else:
                    print(l, trial)
            display(p_events)
            #display(events, len(eeg))


Opening raw data file all_participants/raw_data/participant_2.fif...


/var/folders/0f/x7xs6kjs6k7dfph84ymzzlrr0000gn/T/ipykernel_86248/3794653749.py:4: RuntimeWarning: This filename (all_participants/raw_data/participant_2.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(os.path.join(datadir, f))


Isotrak not found
    Range : 0 ... 2309409 =      0.000 ...  4618.818 secs
Ready.
<Info | 11 non-empty values
 bads: []
 ch_names: Fp1, Fp2, F7, F3, Fz, F4, F8, FC5, FC1, FC2, FC6, T7, C3, Cz, ...
 chs: 64 EEG, 3 misc
 custom_ref_applied: False
 description: An EEG recording
 events: 2439 items (list)
 file_id: 4 items (dict)
 highpass: 0.0 Hz
 lowpass: 250.0 Hz
 meas_date: 2021-09-20 18:53:51 UTC
 meas_id: 4 items (dict)
 nchan: 67
 projs: []
 sfreq: 500.0 Hz
>
[94019     0     1]
[1612049       0       1]


{0: {'start': 54149, 'event': 11, 'end': 56149},
 1: {'start': 56719, 'event': 14, 'end': 58739},
 2: {'start': 59739, 'event': 15, 'end': 61749},
 3: {'start': 62089, 'event': 13, 'end': 64109},
 4: {'start': 64569, 'event': 11, 'end': 66579},
 5: {'start': 67019, 'event': 12, 'end': 69039},
 6: {'start': 69499, 'event': 15, 'end': 71519},
 7: {'start': 71859, 'event': 14, 'end': 73879},
 8: {'start': 74239, 'event': 12, 'end': 76259},
 9: {'start': 76619, 'event': 11, 'end': 78629},
 10: {'start': 79699, 'event': 13, 'end': 81719},
 11: {'start': 82029, 'event': 15, 'end': 84049},
 12: {'start': 84359, 'event': 11, 'end': 86379},
 13: {'start': 86929, 'event': 15, 'end': 88949},
 14: {'start': 88959, 'event': 12, 'end': 90989},
 15: {'start': 90989, 'event': 14, 'end': 93009},
 16: {'start': 93009, 'event': 1, 'end': 95029},
 17: {'start': 95039, 'event': 12, 'end': 97069},
 18: {'start': 97069, 'event': 12, 'end': 99089},
 19: {'start': 99469, 'event': 11, 'end': 101499},
 20: {'sta

In [2]:
raw = mne.io.read_raw_eeglab('./data/p/spoclab/users/szhao/EEG/data/MM05/set_files/Acquisition 232 Data.set')


Reading /Users/Admin/Documents/EEG_speech_gen/data/p/spoclab/users/szhao/EEG/data/MM05/set_files/Acquisition 232 Data.fdt


In [5]:
df = raw.to_data_frame()
df

,time,FP1,FPZ,FP2,AF3,AF4,F7,F5,F3,F1,...,PO3,POZ,PO4,PO6,PO8,CB1,OZ,O2,CB2,O1
0,0.000,0.063756,0.052083,0.079300,0.005805,0.039561,0.033957,0.032724,0.073341,-0.013594,...,0.048977,0.030929,0.044124,0.043304,0.070835,0.000874,0.007362,-0.009491,0.038347,0.038908
1,0.001,-2750.451172,-2265.759277,-3404.814209,-232.794220,-1684.308838,-1473.969116,-1458.195923,-3167.243408,599.331360,...,-2128.986084,-1355.100708,-1916.071289,-1879.783203,-3062.410645,-40.832897,-357.067688,408.193695,-1672.136108,-1695.395630
2,0.002,-5369.527832,-4423.281250,-6646.959961,-454.489807,-3288.162842,-2877.578125,-2846.780273,-6183.165039,1170.027344,...,-4156.239746,-2645.449951,-3740.599121,-3669.756836,-5978.512695,-79.698227,-697.063538,796.896606,-3264.368652,-3309.773682
3,0.003,-7732.736328,-6369.998535,-9572.291016,-654.568298,-4735.320801,-4144.149902,-4099.788086,-8904.371094,1684.953491,...,-5985.359863,-3809.689453,-5386.836426,-5284.816895,-8609.662109,-114.732635,-1003.812134,1147.638794,-4700.981445,-4766.362305
4,0.004,-9734.790039,-8019.169922,-12050.444336,-824.135559,-5961.304687,-5217.292969,-5161.429688,-11209.591797,2121.162598,...,-7534.819336,-4795.927734,-6781.419922,-6652.989258,-10838.606445,-144.361542,-1263.633789,1444.799683,-5917.941895,-6000.235352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2477395,2477.395,-5.297298,-1.966902,-10.517186,1.675127,-7.876565,4.908422,19.323116,-0.520047,2.180877,...,-8.000849,-6.818204,-7.706084,-9.146728,-9.860399,-3.943686,-5.226079,-3.106871,-7.147969,-7.744833
2477396,2477.396,-3.828953,-1.372049,-7.838996,1.354905,-5.998870,4.328710,15.387878,0.126004,1.624173,...,-5.908683,-5.143129,-5.709448,-6.860837,-7.185288,-3.138513,-3.917433,-2.512164,-5.370051,-5.837533
2477397,2477.397,-2.464469,-0.850883,-5.179264,0.954270,-4.033712,3.217353,10.703205,0.356090,1.073886,...,-3.879472,-3.435461,-3.759522,-4.562090,-4.665488,-2.180214,-2.606534,-1.766060,-3.574761,-3.895857
2477398,2477.398,-1.175025,-0.382681,-2.532773,0.496396,-2.004958,1.729323,5.492334,0.292336,0.529000,...,-1.897521,-1.706397,-1.844200,-2.255775,-2.258072,-1.116058,-1.296875,-0.914931,-1.768699,-1.931523


In [18]:
sampling_rate, data = wavfile.read('./data/p/spoclab/users/szhao/EEG/data/MM05/kinect_data/164.wav')


In [41]:
epoch_indices = loadmat('./data/p/spoclab/users/szhao/EEG/data/MM05/e')
epoch_indices['clearing_inds'][0][0][0]

array([ 8233, 13195], dtype=uint16)

In [3]:
raw.info

<Info | 8 non-empty values
 bads: []
 ch_names: FP1, FPZ, FP2, AF3, AF4, F7, F5, F3, F1, FZ, F2, F4, F6, F8, ...
 chs: 62 EEG
 custom_ref_applied: False
 dig: 65 items (3 Cardinal, 62 EEG)
 highpass: 0.0 Hz
 lowpass: 500.0 Hz
 meas_date: unspecified
 nchan: 62
 projs: []
 sfreq: 1000.0 Hz
>